In [8]:
import tensorflow as tf
from tensorflow.keras import utils
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image
import cv2

In [2]:
# Загружаем подготовленные данные 
x_train = np.load('data/x_train.npy')
y_train = np.load('./data/y_train.npy')
x_test = np.load('./data/x_val.npy')
y_test = np.load('./data/y_val.npy')

In [3]:
# Строим модель

def build_model():
    x = tf.keras.Input((256, 256, 3))
    
    out = tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu')(x)
    out1 = tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu')(out)
    out =  tf.keras.layers.MaxPool2D((2,2))(out1)
        
    out = tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu')(out)
    out2 = tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu')(out)
    out =  tf.keras.layers.MaxPool2D((2,2))(out2)
    
        
    out = tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu')(out)
    out3 = tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu')(out)
    out =  tf.keras.layers.MaxPool2D((2,2))(out3)
    
        
    out = tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu')(out)
    out4 = tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu')(out)
    out =  tf.keras.layers.MaxPool2D((2,2))(out4)
    
    out = tf.keras.layers.Conv2D(1024, (3,3), padding='same', activation='relu')(out)
    out = tf.keras.layers.Conv2D(1024, (3,3), padding='same', activation='relu')(out)
    
    out = tf.keras.layers.Conv2DTranspose(512, (3,3), strides=(2,2), padding='same', activation='relu')(out)
    out = tf.concat([out4, out], axis=3)
    
    out = tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu')(out)
    out = tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu')(out)
    
    out = tf.keras.layers.Conv2DTranspose(256, (3,3), strides=(2,2), padding='same', activation='relu')(out)
    out = tf.concat([out3, out], axis=3)
    
    out = tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu')(out)
    out = tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu')(out)
    
    out = tf.keras.layers.Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', activation='relu')(out)
    out = tf.concat([out2, out], axis=3)
    
    out = tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu')(out)
    out = tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu')(out)
    
    out = tf.keras.layers.Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu')(out)
    out = tf.concat([out1, out], axis=3)
    
    out = tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu')(out)
    out = tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu')(out)
    # Выбираем softmax, так как формально класса 2 - пины и фон 
    out = tf.keras.layers.Conv2D(2, (3,3), padding='same', activation='softmax')(out)
    
    return  tf.keras.Model(inputs = x, outputs = out)
    
model = build_model()

In [5]:
# воссоздаем модель 

model = build_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights('./data/U-net/weights.hdf5')

In [6]:
def show_cnt(img):
    
    ttt = img.copy()
    
    plt.figure(figsize=(10,10))
    
    det = model.predict(img[None, ...])[0,...][:,:,1]
    det = np.where(det > 0.1, 255, 0)

    contours, hierarchy = cv2.findContours(
    det.copy(),cv2.RETR_FLOODFILL ,cv2.CHAIN_APPROX_SIMPLE)

    i=0
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            cv2.drawContours(ttt,[cnt],0,(0,255,0),1, cv2.CV_FILLED)
            i+=1
    plt.imshow(ttt)
    
    plt.show()
    
    print(i)
    


    


In [7]:
img = x_test[2]
show_cnt(img)


AttributeError: module 'cv2' has no attribute 'CV_FILLED'

<Figure size 720x720 with 0 Axes>